In [1]:
import pandas as pd
import json, random
from helpers import *
import time
from tqdm import tqdm_notebook
from fa2 import ForceAtlas2
import matplotlib.pyplot as plt
from dateutil import relativedelta
from datetime import timedelta
import glob
from fa2l import force_atlas2_layout

# Load pages and categories

In [2]:
normal_pages = pd.read_pickle("./Data/normal_pages/normal_pages.pkle")

category_pages = pd.read_pickle("./Data/category_pages/category_pages.pkle")

normal_pages.to_pickle("./Data/normal_pages/normal_pages.pkle")

category_pages.to_pickle("./Data/category_pages/category_pages.pkle")

In [4]:
len(normal_pages)

14050827

# Load links

In [4]:
links = pd.read_csv("./Data/diseases/diseases_edges_big.csv",sep='\t',comment='#',header=0,names=['linkSource', 'rel','linkTarget'],dtype=str)

## Filter links if they don't link pages (they link categories)

In [5]:
links_copy = links.copy()

In [6]:
indexes_to_drop = []
for i,row in tqdm_notebook(links_copy.iterrows(),total = links_copy.shape[0]):
    if int(row['linkSource']) in normal_pages.index and int(row['linkTarget']) in normal_pages.index:
        links_copy.at[i,'linkSource'] = normal_pages.loc[int(row['linkSource'])]['article']
        links_copy.at[i,'linkTarget'] = normal_pages.loc[int(row['linkTarget'])]['article']
    else:
        indexes_to_drop.append(i)
        
links_copy = links_copy.drop(links_copy.index[indexes_to_drop])

#Drop 'BELONGS_TO' link

In [7]:
links_copy = links_copy.loc[links['rel'] != 'BELONGS_TO']

In [8]:
links_copy.head()

,linkSource,rel,linkTarget
1,Dutch_hypothesis,LINKS_TO,Allergy
2,Dutch_hypothesis,LINKS_TO,Chronic_obstructive_pulmonary_disease
3,Dutch_hypothesis,LINKS_TO,Asthma
5,Natural_history_of_disease,LINKS_TO,Disease
6,Causes_of_Parkinson\'s_disease,LINKS_TO,Gaucher\'s_disease


## Filter normal pages if they are redirect

In [9]:
normal_pages_copy = normal_pages.copy()

In [10]:
normal_pages_copy = normal_pages_copy.loc[normal_pages_copy['isRedirect'] == False]

## First iterative filter to reduce further number of request

In [11]:
filtered_pages, filtered_links = iterative_filter(normal_pages_copy,links_copy)

In [12]:
len(filtered_links)

227384

In [13]:
len(filtered_links)

22949

## Request to get article activity

In [13]:
s = datetime.datetime(year=2015,month=11,day=1)
e = datetime.datetime(year=2018,month=11,day=1)

In [14]:
timer = time.localtime(time.time())
requests = 0
requests_tot = 0
granularity = 'monthly'
user_activity_dict = {}
to_drop = []

for index,row in tqdm_notebook(filtered_pages.iterrows(),total = filtered_pages.shape[0]):
    requests +=1
    requests_tot+=1
    values = getUserActivity(article=row['article'],granularity=granularity,start=s,end=e)
    
    if (len(values) <= int(monthdelta(s,e) - 1) and granularity == 'monthly') or (len(values) <= int((e - s).days) and granularity == 'daily'):
            
        print("drop "+ row['article'])
        to_drop.append(index)
        filtered_pages.drop(index, inplace=True)
        
    else:
        user_activity_dict[index] = values
        
    #to fit the limit of 100req/s    
    current_time = time.localtime(time.time())
    time_elapsed_sec = current_time[5] - timer[5]
    time_elapsed_min = current_time[4] - timer[4]
    if  time_elapsed_min < 1 and time_elapsed_sec < 0 and requests >= 100:
        time.sleep(max(time_elapsed_sec,0))
        timer = time.localtime(time.time())
        requests = 0
    elif time_elapsed_min >= 1 and time_elapsed_sec >=0:
        timer = time.localtime(time.time())
        requests = 0

drop Congenital_iodine_deficiency_syndrome
drop Huntington\'s_disease
drop Bell\'s_palsy
drop Factitious_disorder_imposed_on_self
drop Wilson\'s_disease
drop Crohn\'s_disease
drop Discredited_HIV/AIDS_origins_theories
drop Cushing\'s_syndrome
drop Lemierre\'s_syndrome
drop Morvan\'s_syndrome
drop Canga\'s_bead_symptom
drop Koch\'s_postulates
drop Ehlers–Danlos_syndromes
drop Ryan_O\'Neal
drop Bowen\'s_disease
drop Human_papillomavirus_infection
drop Binswanger\'s_disease
drop Hirschsprung\'s_disease
drop Alcoholic_Korsakoff_syndrome
drop Gilbert\'s_syndrome
drop Addison\'s_disease
drop Diagnosis_of_HIV/AIDS
drop Cushing\'s_disease
drop Squamous_cell_skin_cancer
drop Orthohantavirus
drop Ja\'far_al-Sadiq
drop McBurney\'s_point
drop Witch\'s_broom
drop Whipple\'s_disease
drop Idiopathic_disease
drop Gaucher\'s_disease
drop Paget\'s_disease_of_bone
drop Writer\'s_cramp
drop Hashimoto\'s_thyroiditis
drop Incidental_imaging_finding
drop Breslow\'s_depth
drop Paget\'s_disease_of_the_breast
d

drop Diaporthe_fraxini-angustifoliae
drop Diaporthe_litchicola
drop Diaporthe_nothofagi
drop Diaporthe_pascoei
drop Diaporthe_salicicola
drop Biphenotypic_sinonasal_sarcoma
drop Candida_auris
drop Type_3c_(Pancreatogenic)_Diabetes
drop Tonsil_carcinoma
drop Tornwaldt\'s_disease
drop Much-Holzmann_reaction
drop Schistosoma_bovis
drop Caleb_Bonham
drop Diana_Walter
drop MORT_(long_non-coding_RNA)
drop Strain_pattern
drop Timeline_of_stroke
drop Estelle_Louise_Jensen
drop Reading_Well_Books_on_Prescription
drop Ernest_Cloquet
drop Woltman_sign
drop Lincoln_sign
drop List_of_pollution-related_diseases
drop Timeline_of_melanoma
drop Timeline_of_leukemia
drop List_of_burn_centres_in_Canada
drop Journal_of_Hospital_Infection
drop CANDLE_syndrome
drop Amafufunyana
drop Chloe_Temtchine
drop Timeline_of_lymphoma
drop Timeline_of_lung_cancer
drop Hemoglobin_H_disease
drop Timeline_of_brain_cancer
drop Premature_thelarche
drop C._V._Subramanian
drop Coalition_for_Epidemic_Preparedness_Innovations


In [15]:
filtered_pages, filtered_links = iterative_filter(filtered_pages,filtered_links)

In [16]:
len(filtered_pages)

7916

In [36]:
filtered_links.loc[43].values

array(['List_of_Washington_Redskins_name_change_advocates', 'LINKS_TO',
       'Washington_Redskins_name_controversy'], dtype=object)

## Create Graph

In [20]:
def createGraph20(nodes, edges):
    """
    Method to create a networkx directed graph from articles node already filtered in categories_filtered
    articles: the wikipedia articles in dataframe ['article']

    categories_filtered: wikipedia articles with their category ['article','categories_filtered'] where 'article' is the index
    categories_filtered have to contain only the node with category you want to appear on the graph.

    links: the hyperlinks between wikipedia articles in form ['linkSource', 'linkTarget']

    return newly created networkx.DiGraph
    """

    #Create graph with networkx
    tup = [[x[0],x[2]] for x in edges.values]
    G = nx.DiGraph()
    G.add_edges_from(tup)
    return G

In [21]:
G = createGraph20(filtered_pages, filtered_links)

## Compute positions with specific layout

In [31]:
forceatlas2 = ForceAtlas2(
                          # Behavior alternatives
                          outboundAttractionDistribution=False,  # Dissuade hubs
                          # Performance
                          jitterTolerance=1.0,  # Tolerance
                          barnesHutOptimize=True,
                          barnesHutTheta=0.5,
                          # Tuning
                          scalingRatio=2.0,
                          strongGravityMode=False,
                          gravity=10.0,
                          # Log
                          verbose=False)

In [32]:
#positions = layout(G, algorithm='ForceAtlas2',fa=forceatlas2)

In [22]:
positions = nx.spring_layout(G,center=[0,0],seed=0)

In [23]:
filtered_pages['coord'] = filtered_pages['article'].map(positions)

## Normalize user activity

In [17]:
user_activity_df = create_df_activity(user_activity_dict)

In [24]:
normalized_user_activity_df = normalize_activity(user_activity_df, factor=40, byColumn=False)

In [25]:
# Create dataframe that contain the mean of each line
user_activity_df_mean = normalized_user_activity_df.copy()
user_activity_df_mean['mean'] = user_activity_df_mean.mean(axis=1)

In [26]:
graph_data = { 'nodes': [], 'edges': [] }

for i,row in filtered_pages.iterrows():
    
    x,y = w_stye_coordinate()
    
    to_be_added = {}
    activity_ind = 0
    for elem in normalized_user_activity_df.loc[i]:
        to_be_added[str(activity_ind)+"_size"] = elem
        activity_ind +=1
        #to_be_added[str(activity_ind)+"_size"] = 0 if(elem[0]<=0) else(np.log(elem[0]))
    to_be_added['label'] = row['article']
    to_be_added['x'] = x
    to_be_added["y"] = y
    to_be_added["id"]= row['article']
    to_be_added["attributes"]= {"category":'Unknown'}
    to_be_added["user_activity"]= user_activity_dict[i]
    to_be_added["color"]= 'rgb(30,30,30)'
    to_be_added["originalColor"]= 'rgb(120,30,30)'
    #"size": 0 if(user_activity_dict[i][0][0]==0) else (np.log(user_activity_dict[i][0][0]))
    to_be_added["size"]= 10
    to_be_added["square_x"]= x
    to_be_added["square_y"]= y
    to_be_added["square_size"]= 10
    to_be_added["square_color"]= 'rgb(30,30,30)'
    to_be_added["correct_x"]= row['coord'][0]
    to_be_added["correct_y"]= row['coord'][1]
    to_be_added["correct_size"]= normalized_user_activity_df.loc[i][0]
    to_be_added["activity_mean"]= user_activity_df_mean.loc[i]['mean']
    #to_be_added["correct_color"]= row['color']
    to_be_added["correct_color"]= 'rgb(120,30,30)'
    graph_data['nodes'].append(to_be_added)

for i,row in filtered_links.iterrows():
    graph_data['edges'].append({
            "id": str(i),
            "source": row['linkSource'],
            "target": row['linkTarget'],
            "attributes" : {},
            "color": 'rgb(30,30,30)',
            "originalColor": 'rgb(30,30,30)',
            #"color": row['color'],
            #"originalColor": row['color'],
            "size": 0.1,
            "type": 'curve'
        })

In [28]:
save_graph_json('./sigma_jup_viz/diseases_3depth.json',graph_data)

In [29]:
filtered_pages.to_pickle("./Data/diseases/deep_filtered_pages_"+s.strftime("%Y-%m-%d")+"_"+e.strftime("%Y-%m-%d")+"_"+granularity+".pkle")

In [30]:
filtered_links.to_pickle("./Data/diseases/deep_filtered_links_"+s.strftime("%Y-%m-%d")+"_"+e.strftime("%Y-%m-%d")+"_"+granularity+".pkle")

In [32]:
len(filtered_links)

197663